# Optimal GF for lake

1. Determine the frequency $u$ for which the average FSC progressively computed from the normalized frequency 0.5 is higher than some given threshold $t=0.143$, by default.
2. Compute the corresponding Gaussian filter length $\tau$ whose cut-off frequency is $u$.
3. Filter the image.

Hay que estudiar:
1. Si el valle en la curva SFRC es culpa del filtro separable (comparar con el 2D puro). No es culpa. Por encima de la frecuencia normalizada 0.25 la SFRC no es significativa.
2. Si usando random shuffling y OF es posible obtener una SFRC mejor.

In [ ]:
import time
from collections import namedtuple

try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

import scipy.ndimage

try:
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker
except:
    !pip install matplotlib
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker

try:
    from skimage import io as skimage_io
except:
    !pip install scikit-image
    from skimage import io as skimage_io

try:
    import information_theory as IT
except:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
    import information_theory as IT

import utils        #freq, c_avg = fsc.get_SFRC_curve(denoised)
        #first_half = c_avg[:len(c_avg)>>1]
        #SFRC_curve.append(first_half)
        #plt.imshow(denoised, cmap="gray")
        #plt.show()
        #input()

In [ ]:
# apt install cm-super-minimal
# apt install dvipng
plt.rcParams.update({
    "text.usetex": True,
    #"font.family": "Helvetica",
    "font.family": "Serif",
    "text.latex.preamble": r"\usepackage{amsmath} \usepackage{amsfonts}"
})

In [ ]:
import logging
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.WARNING)

In [ ]:
try:
    from self_fourier_shell_correlation import fsc_utils as fsc
except:
    !pip install "self_fourier_shell_correlation @ git+https://github.com/vicente-gonzalez-ruiz/self_fourier_shell_correlation"
    from self_fourier_shell_correlation import fsc_utils as fsc

In [ ]:
try:
    import denoising.image.gaussian as denoising
except:
    !pip install "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
    import denoising.image.gaussian as denoising

In [ ]:
denoiser = denoising.Monochrome_Denoising(logger)

### Inputs

In [ ]:
#Args = namedtuple("args", ["input"])
#args = Args("lake_0MMPG.png")
Args = namedtuple("args", ["X", "Y"])
args = Args("http://www.hpca.ual.es/~vruiz/images/lake.png", "lake_0MMPG.png")
#args = Args("http://www.hpca.ual.es/~vruiz/images/lake.png", "lake_0MMPG.png")

In [ ]:
X = skimage_io.imread(args.X)

In [ ]:
try:
    Y = skimage_io.imread(args.Y)
except FileNotFoundError:
    %run lake_0MMPG.ipynb
    Y = skimage_io.imread(args.Y)

In [ ]:
utils.imshow(X)

In [ ]:
utils.imshow(Y)

### Compute SFRC

In [ ]:
freq_1, c_avg_Y_1 = fsc.get_SFRC_curve__even_odd(X)
freq_2, c_avg_Y_2 = fsc.get_SFRC_curve__even_odd(Y)

from scipy.ndimage import gaussian_filter1d

plt.plot(freq_1, c_avg_Y_1, label="X")
filtered_c_avg_Y_1 = gaussian_filter1d(c_avg_Y_1, sigma=5)
plt.plot(freq_1, filtered_c_avg_Y_1)

plt.plot(freq_2, c_avg_Y_2, label="Y")
filtered_c_avg_Y_2 = gaussian_filter1d(c_avg_Y_2, sigma=5)
plt.plot(freq_2, filtered_c_avg_Y_2)

plt.legend()

In [ ]:
freq_1, c_avg_Y_1 = fsc.get_SFRC_curve__SPRS(X, std_dev=5)
freq_2, c_avg_Y_2 = fsc.get_SFRC_curve__SPRS(Y, std_dev=5)

from scipy.ndimage import gaussian_filter1d

plt.plot(freq_1, c_avg_Y_1, label="X")
filtered_c_avg_Y_1 = gaussian_filter1d(c_avg_Y_1, sigma=5)
plt.plot(freq_1, filtered_c_avg_Y_1)

plt.plot(freq_2, c_avg_Y_2, label="Y")
filtered_c_avg_Y_2 = gaussian_filter1d(c_avg_Y_2, sigma=5)
plt.plot(freq_2, filtered_c_avg_Y_2)

plt.legend()

In [ ]:
std_devs = [i for i in range(0, 200, 40)]
poisson_ratio = 0.5
gamma = 0.15
for std_dev in std_devs: # Number of noise levels
    PCC_curve = []
    Y = utils.generate_MPGN(X, std_dev, gamma, poisson_ratio).reshape(X.shape)
    print("std_dev =", std_dev)
    freq, c_avg = fsc.get_SFRC_curve__even_odd(Y)
    plt.plot(freq, c_avg, label=rf"$\sigma={std_dev}$")
plt.legend()

In [ ]:
std_devs = [i for i in range(0, 200, 40)]
poisson_ratio = 0.5
gamma = 0.15
for std_dev in std_devs: # Number of noise levels
    PCC_curve = []
    Y = utils.generate_MPGN(X, std_dev, gamma, poisson_ratio).reshape(X.shape)
    print("std_dev =", std_dev)
    freq, c_avg = fsc.get_SFRC_curve__SPRS(Y)
    plt.plot(freq, c_avg, label=rf"$\sigma={std_dev}$")
plt.legend()

In [ ]:
std_devs = [i for i in range(0, 200, 40)]
poisson_ratio = 0.5
gamma = 0.15
for std_dev in std_devs: # Number of noise levels
    PCC_curve = []
    Y = utils.generate_MPGN(X, std_dev, gamma, poisson_ratio).reshape(X.shape)
    print("std_dev =", std_dev)
    freq, c_avg = fsc.get_SFRC_curve__chessboard(Y)
    plt.plot(freq, c_avg, label=rf"$\sigma={std_dev}$")
plt.legend()

In [ ]:
std_devs = [i for i in range(0, 200, 40)]
poisson_ratio = 0.5
gamma = 0.15
for std_dev in std_devs: # Number of noise levels
    PCC_curve = []
    Y = utils.generate_MPGN(X, std_dev, gamma, poisson_ratio).reshape(X.shape)
    print("std_dev =", std_dev)
    freq, c_avg = fsc.get_SFRC_curve__interpolated_chessboard(Y)
    plt.plot(freq, c_avg, label=rf"$\sigma={std_dev}$")
plt.legend()

In [ ]:
def estimate_noise(signal):
    noise_estimation = signal[len(signal)-1]
    return noise_estimation
    
def find_cutoff_freq__EO(image, beta=0.5):
    freq, c_avg = fsc.get_SFRC_curve__even_odd(image)
    #c_avg = c_avg[0:len(c_avg)>>1]
    filtered_c_avg = gaussian_filter1d(c_avg, 5)
    #min_curve = np.abs(np.min(filtered_c_avg)); cutoff_idx = np.where(filtered_c_avg < min_curve * (1 + beta))[0][0]
    #noise_estimation = estimate_noise(filtered_c_avg); cutoff_idx = np.where(filtered_c_avg < noise_estimation ** beta )[0][0]
    #noise_estimation = estimate_noise(filtered_c_avg); cutoff_idx = np.where(filtered_c_avg < noise_estimation + beta)[0][0]
    cutoff_idx = np.where(filtered_c_avg < filtered_c_avg[0] * beta)[0][0]
    cutoff_freq = freq[cutoff_idx>>1]
    #return (cutoff_freq - 0.05) * 3
    return (cutoff_freq) * 2 # Only half of the frequencies are meaninful

def find_cutoff_freq__RS(image, beta=0.2, std_dev=5):
    freq, c_avg = fsc.get_SFRC_curve__SPRS(image, std_dev)
    #c_avg = c_avg[0:len(c_avg)>>1]
    #freq = freq[0:len(freq)>>1]
    #freq /= 2
    #freq, c_avg = fsc.get_SFRC_curve_EO(image)
    filtered_c_avg = gaussian_filter1d(c_avg, 5)
    #min_curve = np.abs(np.min(filtered_c_avg)); cutoff_idx = np.where(filtered_c_avg < min_curve + beta)[0][0]
    #min_curve = np.abs(np.min(filtered_c_avg)); cutoff_idx = np.where(filtered_c_avg < min_curve * (1 + beta))[0][0]
    cutoff_idx = np.where(filtered_c_avg < filtered_c_avg[0] * beta)[0][0]
    #noise_estimation = estimate_noise(filtered_c_avg); cutoff_idx = np.where(filtered_c_avg < noise_estimation ** beta )[0][0]
    #noise_estimation = estimate_noise(filtered_c_avg); cutoff_idx = np.where(filtered_c_avg < noise_estimation + beta)[0][0]
    cutoff_freq = freq[cutoff_idx>>0]
    #return (cutoff_freq - 0.070) * 2.5 #  Pendiente, 1/umbral
    return (cutoff_freq - 0.09) * 5 # See pure_noise_SFRC.ipynb

In [ ]:
def convert_to_tau(eta):
    return 0.141/(eta)

In [ ]:
poisson_ratio = 0.5
gamma = 0.15
std_dev = 160
Y = utils.generate_MPGN(X, std_dev, gamma, poisson_ratio).reshape(X.shape) 

In [ ]:
eta_X = find_cutoff_freq__EO(X)
eta_Y = find_cutoff_freq__EO(Y)
print(eta_X)
print(eta_Y)

In [ ]:
tau_X = convert_to_tau(eta_X)
tau_Y = convert_to_tau(eta_Y)
print(tau_X)
print(tau_Y)

In [ ]:
eta_X = find_cutoff_freq__RS(X)
eta_Y = find_cutoff_freq__RS(Y)
print(eta_X)
print(eta_Y)

In [ ]:
tau_X = convert_to_tau(eta_X)
tau_Y = convert_to_tau(eta_Y)
print(tau_X)
print(tau_Y)

### lake_GF_estimation

In [ ]:
def find_taus_EO():
    std_devs = [i for i in range(0, 200, 40)]
    PCC_curves = []
    poisson_ratio = 0.5
    gamma = 0.15
    taus_EO = []
    for std_dev in std_devs: # Number of noise levels
        PCC_curve = []
        Y = utils.generate_MPGN(X, std_dev, gamma, poisson_ratio).reshape(X.shape)
        eta = find_cutoff_freq__EO(Y)
        tau = convert_to_tau(eta)
        taus_EO.append(tau)
        #sigma = np.array([tau, tau])
        #kernel = [None]*2
        #kernel[0] = utils.get_gaussian_kernel(tau)
        #kernel[1] = utils.get_gaussian_kernel(tau)
        #denoised = denoiser.filter(Y, kernel)
        print("std_dev =", std_dev, "tau =", tau)
        #freq, c_avg = fsc.get_SFRC_curve__even_odd(denoised)
        #freq, c_avg = fsc.get_SFRC_curve_EO(denoised)
        #freq, c_avg = fsc.get_SFRC_curve_EO(Y)
        #freq, c_avg = fsc.get_SFRC_curve_RS(Y, std_dev=3)
        #plt.plot(freq, c_avg, label=f"tau={tau}")
    #plt.legend()
    return taus_EO

In [ ]:
N = 2

In [ ]:
taus_EO = []
for i in range(N):
    taus_EO.append(find_taus_EO())

In [ ]:
def average_lists(lists):
    """
    Computes a list where each element is the average of corresponding elements
    from a list of lists.

    Args:
        lists: A list of lists, where each inner list has the same length.

    Returns:
        A list containing the averages, or an empty list if the input is invalid.
    """

    if not lists:
        return []  # Return empty if the input list is empty

    num_lists = len(lists)
    if num_lists == 0:
      return []

    if not all(isinstance(lst, list) for lst in lists):
      return [] #return empty if not all the elements of the main list are lists

    if not all(len(lst) == len(lists[0]) for lst in lists):
        return []  # Return empty if inner lists have different lengths

    num_elements = len(lists[0])
    avg_list = []

    for j in range(num_elements):
        total = 0
        for i in range(num_lists):
            total += lists[i][j]
        avg_list.append(total / num_lists)

    return avg_list

In [ ]:
avg_taus_EO = average_lists(taus_EO)

In [ ]:
def find_taus_RS():
    std_devs = [i for i in range(0, 200, 40)]
    PCC_curves = []
    poisson_ratio = 0.5
    gamma = 0.15
    std_dev_RS = 3
    taus_RS = []
    for std_dev in std_devs: # Number of noise levels
        PCC_curve = []
        Y = utils.generate_MPGN(X, std_dev, gamma, poisson_ratio).reshape(X.shape)
        eta = find_cutoff_freq__RS(Y, std_dev=std_dev_RS)
        tau = convert_to_tau(eta)
        taus_RS.append(tau)
        #sigma = np.array([tau, tau])
        #kernel = [None]*2
        #kernel[0] = utils.get_gaussian_kernel(tau)
        #kernel[1] = utils.get_gaussian_kernel(tau)
        #denoised = denoiser.filter(Y, kernel)
        print("std_dev =", std_dev, "tau =", tau)
        #freq, c_avg = fsc.get_SFRC_curve__random_shuffling(denoised, std_dev=std_dev_RS)
        #freq, c_avg = fsc.get_SFRC_curve_EO(denoised)
        #freq, c_avg = fsc.get_SFRC_curve_EO(Y)
        #freq, c_avg = fsc.get_SFRC_curve_RS(Y, std_dev=3)
        #plt.plot(freq, c_avg, label=f"tau={tau}")
    #plt.legend()
    return taus_RS

In [ ]:
taus_RS = []
for i in range(N):
    taus_RS.append(find_taus_RS())

In [ ]:
avg_taus_RS = average_lists(taus_RS)

In [ ]:
optimal_taus = np.load("lake_GF_optimal_taus.npy")
optimal_taus = optimal_taus.tolist()

In [ ]:
x_labels = [r"$\sigma=0$", r"$\sigma=20$", r"$\sigma=40$", r"$\sigma=60$", r"$\sigma=80$"]
plt.plot(x_labels, avg_taus_EO, label="EO")
plt.plot(x_labels, avg_taus_RS, label="RS")
plt.plot(optimal_taus, label="optimal")
plt.xlabel(r"MPG noise level ($\sigma, \gamma=0.15$)")
plt.ylabel(r"$\tau$")
plt.legend()
plt.savefig("lake_GF_estimation.pdf", bbox_inches='tight')

In [ ]:
1/np.sqrt(2)